In [1]:
import pandas as pd
import numpy as np

from goatools.base import download_ncbi_associations
from goatools.associations import read_ncbi_gene2go
import sys
from goatools.associations import read_gaf
from goatools.base import dnld_gaf

In [2]:
save_path = "/media/DATA/serbulent/DATA/Thesis/ReviewPaper/preprocess/go_related_data/"

In [3]:
go_dataframe = pd.read_pickle(save_path + 'go_dataframe.pkl')

In [4]:
go_dataframe

,GO_ID,Term_Level,Protein_Number,Number_Category,Term_Specificity,Aspect
0,GO:1903494,4,1,insufficient,Shallow,biological_process
1,GO:0019166,5,6,Low,Normal,molecular_function
2,GO:0033273,4,18,Low,Shallow,biological_process
3,GO:0001079,5,1,insufficient,Normal,biological_process
4,GO:1900027,6,26,Low,Normal,biological_process
...,...,...,...,...,...,...
20359,GO:0009070,6,27,Low,Normal,biological_process
20360,GO:0001534,2,5,Low,Shallow,cellular_component
20361,GO:0071901,8,126,Middle,Specific,biological_process
20362,GO:0061317,7,2,insufficient,Normal,biological_process


In [5]:
go_network = pd.read_csv("/media/DATA/serbulent/DATA/Thesis/ReviewPaper/preprocess/go_related_data/GO.cytoscape.sif",sep="\t", header=None)

In [6]:
go_network.columns  = ["node1","interaction","node2"]

In [7]:
go_human_propagated_set = set(go_dataframe['GO_ID'])

In [8]:
go_human_propagated_set_min_5 = set(go_dataframe[go_dataframe['Protein_Number'] >= 5]['GO_ID'])

In [9]:
set(go_network['interaction'])

{'disjoint_from:',
 'ends_during',
 'happens_during',
 'has_part',
 'is_a:',
 'negatively_regulates',
 'occurs_in',
 'part_of',
 'positively_regulates',
 'property_value:',
 'regulates'}

In [10]:
go_network_interaction_filtered = go_network[(go_network['interaction'] =='is_a:') |  (go_network['interaction'] =='part_of:')]

In [11]:
go_network_interaction_filtered_human = go_network_interaction_filtered\
[go_network_interaction_filtered['node1'].isin(go_human_propagated_set) & \
 go_network_interaction_filtered['node2'].isin(go_human_propagated_set)]

In [12]:
go_network_interaction_filtered_human

,node1,interaction,node2
2,GO:0000002,is_a:,GO:0007005
3,GO:0000003,is_a:,GO:0008150
8,GO:0000010,is_a:,GO:0016765
11,GO:0000012,is_a:,GO:0006281
12,GO:0000014,is_a:,GO:0004520
...,...,...,...
94525,GO:2001306,is_a:,GO:1901617
94526,GO:2001306,is_a:,GO:2001301
94527,GO:2001306,is_a:,GO:2001304
94554,GO:2001311,is_a:,GO:0006650


In [13]:
go_network_interaction_filtered_human.to_csv(save_path + "go_network_interaction_filtered_human.sif",sep="\t",header=False, index=False)

In [14]:
go_network_interaction_filtered_human_min_5 = go_network_interaction_filtered\
[go_network_interaction_filtered['node1'].isin(go_human_propagated_set_min_5) & \
 go_network_interaction_filtered['node2'].isin(go_human_propagated_set_min_5)]

In [15]:
go_network_interaction_filtered_human_min_5

,node1,interaction,node2
2,GO:0000002,is_a:,GO:0007005
3,GO:0000003,is_a:,GO:0008150
11,GO:0000012,is_a:,GO:0006281
12,GO:0000014,is_a:,GO:0004520
17,GO:0000018,is_a:,GO:0051052
...,...,...,...
94506,GO:2001300,is_a:,GO:0006690
94507,GO:2001300,is_a:,GO:0033559
94508,GO:2001301,is_a:,GO:0006636
94509,GO:2001301,is_a:,GO:0046456


In [16]:
go_network_interaction_filtered_human_min_5.to_csv(save_path + "go_network_interaction_filtered_human_min_5.sif",sep="\t",header=False, index=False)

In [17]:
go_number_df = go_dataframe[['GO_ID', 'Protein_Number']]

In [18]:
go_number_df['protein_number_log2'] = np.log2(list(go_dataframe['Protein_Number'])) * 40

/media/DATA/serbulent/anaconda3/envs/embedding_similarity/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
go_number_df['protein_number_log10'] = np.log10(list(go_dataframe['Protein_Number'])) * 40

/media/DATA/serbulent/anaconda3/envs/embedding_similarity/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
dissimilar_terms_dataframe_save_path = save_path + 'dissimilar_terms_dataframe.pkl'
dissimilar_terms_dataframe = pd.read_pickle(dissimilar_terms_dataframe_save_path)

In [21]:
pfp_check_list = list(go_number_df['GO_ID'].isin(dissimilar_terms_dataframe['GO_ID']))

In [22]:
go_number_df['in_benchmark'] = ["PFP" if pfp else False for pfp in pfp_check_list]

/media/DATA/serbulent/anaconda3/envs/embedding_similarity/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
go_number_df

,GO_ID,Protein_Number,protein_number_log2,protein_number_log10,in_benchmark
0,GO:1903494,1,0.000000,0.000000,False
1,GO:0019166,6,103.398500,31.126050,False
2,GO:0033273,18,166.797000,50.210900,False
3,GO:0001079,1,0.000000,0.000000,False
4,GO:1900027,26,188.017589,56.598934,False
...,...,...,...,...,...
20359,GO:0009070,27,190.195500,57.254551,False
20360,GO:0001534,5,92.877124,27.958800,False
20361,GO:0071901,126,279.091197,84.014822,False
20362,GO:0061317,2,40.000000,12.041200,False


# Add Protein Semantic Similarity Task GO Terms

In [24]:
uniprot_metadata_directory = "/media/DATA/serbulent/DATA/Thesis/ReviewPaper/Uniprot/"
uniprot_metadata_file_path = uniprot_metadata_directory + "uniprot_human_all.tab"
uniprot_vars = ['Entry','Entry name','Status','Protein names','Gene names','Organism','Length','Annotation' ]
uniprot_df = pd.read_csv(uniprot_metadata_file_path, sep='\t', header=None, names=uniprot_vars)
human_proteins = set(uniprot_df['Entry'])

In [25]:
sim_matrix_500_MF = "/media/DATA/serbulent/DATA/Thesis/ReviewPaper/preprocess/semantic_similarity_preprocess_results/human_MF_proteinSimilarityMatrix_for_highest_annotated_500_proteins.csv"
sim_matrix_500_BP = "/media/DATA/serbulent/DATA/Thesis/ReviewPaper/preprocess/semantic_similarity_preprocess_results/human_BP_proteinSimilarityMatrix_for_highest_annotated_500_proteins.csv"
sim_matrix_500_CC = "/media/DATA/serbulent/DATA/Thesis/ReviewPaper/preprocess/semantic_similarity_preprocess_results/human_CC_proteinSimilarityMatrix_for_highest_annotated_500_proteins.csv"

human_proteinSimilarityMatrix_MF = pd.read_csv(sim_matrix_500_MF)
human_proteinSimilarityMatrix_BP = pd.read_csv(sim_matrix_500_BP)
human_proteinSimilarityMatrix_CC = pd.read_csv(sim_matrix_500_CC)

sim_MF_list = list(human_proteinSimilarityMatrix_MF.columns)
sim_BP_list = list(human_proteinSimilarityMatrix_BP.columns)
sim_CC_list = list(human_proteinSimilarityMatrix_CC.columns)

sim_protein_list = sim_MF_list + sim_BP_list + sim_CC_list

In [26]:
len(set(sim_protein_list))

1123

In [27]:
fin_gaf = "/media/DATA/serbulent/DATA/Thesis/ReviewPaper/preprocess/semantic_similarity_preprocess_inputs/goa_human.gaf"

geneid2gos_human_MF = read_gaf(fin_gaf,namespace='MF', ev_exclude={'IEA'})
geneid2gos_human_BP = read_gaf(fin_gaf,namespace='BP', ev_exclude={'IEA'})
geneid2gos_human_CC = read_gaf(fin_gaf,namespace='CC', ev_exclude={'IEA'})

HMS:0:00:07.265092 482,898 annotations READ: /media/DATA/serbulent/DATA/Thesis/ReviewPaper/preprocess/semantic_similarity_preprocess_inputs/goa_human.gaf 
15493 IDs in loaded association branch, MF
HMS:0:00:07.189001 482,898 annotations READ: /media/DATA/serbulent/DATA/Thesis/ReviewPaper/preprocess/semantic_similarity_preprocess_inputs/goa_human.gaf 
15198 IDs in loaded association branch, BP
HMS:0:00:06.559537 482,898 annotations READ: /media/DATA/serbulent/DATA/Thesis/ReviewPaper/preprocess/semantic_similarity_preprocess_inputs/goa_human.gaf 
16254 IDs in loaded association branch, CC


In [28]:
go_mf_set = set()
for protein_id in sim_MF_list:
    go_mf_set.update(geneid2gos_human_MF.get(protein_id))

go_bp_set = set()
for protein_id in sim_BP_list:
    go_bp_set.update(geneid2gos_human_BP.get(protein_id))
    
go_cc_set = set()
for protein_id in sim_CC_list:
    go_cc_set.update(geneid2gos_human_CC.get(protein_id))
    
go_all_set = set()
go_all_set.update(go_mf_set)
go_all_set.update(go_bp_set)
go_all_set.update(go_cc_set)

In [29]:
len(go_all_set)

7398

In [30]:
sim_check_list = list(go_number_df['GO_ID'].isin(go_all_set))

In [31]:
in_benchmark_list = list(go_number_df['in_benchmark'])
i = 0
for sim in sim_check_list:
    if sim:
        in_benchmark_list[i] = "SSM"
    i+=1
go_number_df['in_benchmark'] = in_benchmark_list

In [35]:
len(pfp_check_list)

20364

In [40]:
# PFP should overrride others
in_benchmark_list = list(go_number_df['in_benchmark'])
i = 0
for sim in pfp_check_list:
    if sim:
        in_benchmark_list[i] = "PFP"
    i+=1
go_number_df['in_benchmark'] = in_benchmark_list

117


/media/DATA/serbulent/anaconda3/envs/embedding_similarity/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [41]:
go_number_df.to_csv(save_path + "go_annotation_numbers.tsv",sep="\t",index=False)

117